In [1]:
import sys
import os
sys.path.append(os.path.abspath("src"))

In [2]:
%pwd

'c:\\Users\\reda-\\Desktop\\Projects\\MLOPS-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\reda-\\Desktop\\Projects\\MLOPS-Project'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath =  CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath =  SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from MLProject import logger
from MLProject.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! With following info :  \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingetion = DataIngestion(config=data_ingestion_config)
    data_ingetion.download_file()
    data_ingetion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-02 18:27:33,170: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-02 18:27:33,173: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-02 18:27:33,175: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-02 18:27:33,177: INFO: common: created directory at: artifacts]
[2025-10-02 18:27:33,177: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-02 18:27:34,096: INFO: 1387445425: artifacts/data_ingestion/data.zip downloaded! With following info :  
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A7D0:1CC011:1CE674:20AECD:68DEB604
Accept-Ranges: bytes
Date: Thu